In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pydicom

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.widgets import Slider

from IPython.display import HTML

We take one of the patients and load all its dicom images as numpy arrays. 

In [ ]:
?np.sort

In [ ]:
patient_id = "ID00035637202182204917484"

dicom_path = "/kaggle/input/osic-pulmonary-fibrosis-progression/train"

files = np.array([f.replace(".dcm","") for f in os.listdir(f"{dicom_path}/{patient_id}/")])
files = -np.sort(-files.astype("int"))
dicoms = [f"{dicom_path}/{patient_id}/{f}.dcm" for f in files]

In these Dicoms the slope and intercept seems to be 1 and 0 respectively so there is no need for the transformation, but putting the whole code just in case and also for recyclability!

In [ ]:
images = []
for dcm in dicoms:
    tmp = pydicom.dcmread(dcm)
    slope = tmp.RescaleSlope
    intercept = tmp.RescaleIntercept
    final = tmp.pixel_array*slope + intercept
    images.append(final)
    
images = np.array(images) 

## From the top

Here is how to create an animation out of an array of images.

In [ ]:
fig = plt.figure()

ims = []
for image in range(0,images.shape[0],10):
    im = plt.imshow(images[image,:,:], 
                    animated=True, cmap=plt.cm.bone)
    plt.axis("off")
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=False,
                                repeat_delay=1000)

plt.close()

Two ways of animating the images! 
The JavaScript one is way more iteractive while the second html5 one is basically a video.

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
HTML(ani.to_html5_video())

## Frontwise

In [ ]:
fig = plt.figure()

ims = []
for image in range(0,images.shape[1],5):
    im = plt.imshow(images[:,image,:], animated=True, cmap=plt.cm.bone)
    plt.axis("off")
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=False,
                                repeat_delay=1000)

plt.close()

HTML(ani.to_jshtml())

## Sidewise

In [ ]:
fig = plt.figure()

ims = []
for image in range(0,images.shape[2],5):
    im = plt.imshow(images[:,:,image], animated=True, cmap=plt.cm.bone)
    plt.axis("off")
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=False,
                                repeat_delay=1000)

plt.close()

HTML(ani.to_jshtml())

From the internet I found that in a CT Scan, the HU (Hounsfield's units) is calcluated based on the linear attenuation coefficient $\mu$, with the following fomula:

$$
HU = 1000 \cdot \frac{\mu_{X} - \mu_{water}}{\mu_{air} - \mu_{water}}
$$

And the most common values are:

|Substance       |  	 HU      |
|----------------|---------------|
|Air 	         |    -1000      |
|Lung 	         |    -700       |
|Fat 	         |    -84        |
|Water 	         |     0         |
|CSF 	         |     15        |
|Blood 	         | +30 to +45    |
|Muscle          |	+40          |
|Soft Tissue     | 	+100 to +300 |
|Cancellous Bone | 	+700         |
|Dense Bone 	 |  +3000        |

While -2048 would indicate a missing value (since the scan has a circular shape while our plot is rectangular.

Here is the histogram of the CT scan of the Patient in consideration.

In [ ]:
plt.hist(np.array(images).reshape(-1,), bins=50)
plt.show()

Unfortunately this last part won't work in the kaggle view mode, but on edit mode of the notebook. 
But truth to be told the the java script version above can be seen as a interactive mode, where you can go frame by frame anche see all the slices.

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
%matplotlib notebook

fig = plt.figure(figsize=(5,5))

img_plot = plt.imshow(images[0], cmap="Greys")
plt.axis("off")

@interact(slice = widgets.IntSlider(min=0, max=len(images), step=1, value=0))
def update(slice):
    global img_plot
    img_plot.set_data(images[int(slice)])
    plt.draw()